In [1]:
# error
# prompt -> pip install protobuf==3.20.*

### Import

In [1]:
from tensorflow.keras.models import load_model

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from glob import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from gtts import gTTS
from IPython.display import Audio

In [4]:
import os
import io
 
import cv2
from google.cloud import vision

### Define

In [5]:
def sound():
    display(Audio('alarm-no3.mp3', autoplay=True))

In [6]:
def get_narrator(text):
    tts = gTTS(text=text, lang='ko')
    filename = 'narrator.mp3'
    tts.save(filename)
    
    display(Audio(filename, autoplay=True))
    
# get_narrator("어린이 보호구역입니다")

In [7]:
def get_text(path):
    result = []
    
    # Load image
    with io.open(path, 'rb') as f:
        content = f.read()
    
    # Get text from image
    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    for text in texts[1:]:
        if text.description.isdigit():
            if int(text.description) in [x for x in range(20, 90, 10)]:
                result.append(text.description)
        elif text.description=='어린이':
            result.append(text.description)
            
    return result

In [8]:
def predict_road(model):
    df = pd.DataFrame({'image_path': glob('capture/*.jpg')})
    
    test_datagen = ImageDataGenerator(rescale=1/255.)
    test_generator = test_datagen.flow_from_dataframe(
        df,
        x_col='image_path',
        y_col='None',
        target_size=(250, 250),
        class_mode=None,
        shuffle=False
    )
    
    pred = model.predict_generator(test_generator)
    
    return np.argmax(pred, axis=-1)

In [9]:
def alert_speed_limit(texts):
    message = []
    digit_boolean = list(map(lambda x: x.isdigit(), texts))
    if np.all(digit_boolean):  # 모두 숫자
        for text in texts:
            message.append(f"제한속도 시속 {text}km 구간입니다")
    elif np.any(digit_boolean):  # '어린이' 하나 이상 포함
        for text in np.array(texts)[digit_boolean]:
            message.append(f"어린이 보호구역입니다. 시속 {text}km로 주행하세요")
    else:
        message.append("어린이 보호구역입니다")
        
    for m in message:
        get_narrator(m)

### Prepare

In [10]:
# Use google API
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'JSON_FILENAME.json'
 
client_options = {'api_endpoint': 'eu-vision.googleapis.com'}
client = vision.ImageAnnotatorClient(client_options=client_options)

In [11]:
# Load model
model = load_model('MODEL_NAME.h5')

In [12]:
# Connect Webcam
url = "http://IP_ADDRESS:8080/video"
cap = cv2.VideoCapture(url)

fps = cap.get(cv2.CAP_PROP_FPS)  # fps = 25.0

In [13]:
# Empty capture folder
for f in glob('capture/*.jpg'):
    os.remove(f)

In [14]:
# Setting default text
passibility = "Passibility"
color = (255, 255, 255)

### Run

In [15]:
L = []  # For counting impassible road
while(True):
    camera, frame = cap.read()

    # # frame resize
    # resize_frame = cv2.resize(frame, (250, 250), interpolation=cv2.INTER_AREA)

    # frame crop & resize
    resize_frame = cv2.resize(frame[:, 420:1500], (250, 250), interpolation=cv2.INTER_AREA)

    if resize_frame is not None:
        
        # Capture 5 images per 1 second
        if (int(cap.get(1)) % 5 == 0):
            # Save image as 001 ~ 005.jpg
            filename = f'{str(len(os.listdir("capture"))%5 + 1).zfill(3)}.jpg'
            cv2.imwrite(f'capture/{filename}', resize_frame)
            
            # Get text from image (per 5 seconds)
            if (int(cap.get(1)) % 125 == 0):  # 5 * fps
                if get_text(f'capture/{filename}'):
                    alert_speed_limit(get_text(f'capture/{filename}'))
                                        
        if len(os.listdir('capture')) >= 5:
            pred = predict_road(model)

            if np.all(pred == 1):
                passibility = "Passible"
                color = (255, 0, 0)
            elif np.all(pred == 0):
                passibility = "Impassible"
                color = (0, 0, 255)
                L.append(np.all(pred == 0))  # Append 1 if all of predictions 0
                
            for f in glob('capture/*.jpg'):
                os.remove(f)
                
        cv2.putText(resize_frame, passibility, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.imshow("Frame", resize_frame)

    # Detect impassible road more than 5 times -> sound()
    if L.count(1) > 4:
        sound()
        break
    
    # To finish program(close windows), press q                     
    q = cv2.waitKey(1)
    if q == ord("q"):
        break
cv2.destroyAllWindows()

Found 5 validated image filenames.


C:\Users\wbs04\AppData\Local\Temp\ipykernel_27640\2587802143.py:14: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred = model.predict_generator(test_generator)


Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.


Found 5 validated image filenames.


C:\Users\wbs04\AppData\Local\Temp\ipykernel_27640\2587802143.py:14: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred = model.predict_generator(test_generator)


...

C:\Users\wbs04\AppData\Local\Temp\ipykernel_27640\2587802143.py:14: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred = model.predict_generator(test_generator)


Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated im

Found 5 validated image filenames.


C:\Users\wbs04\AppData\Local\Temp\ipykernel_27640\2587802143.py:14: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred = model.predict_generator(test_generator)


Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
